# Install dependencies

In [3]:
!apt-get install -y xvfb python3-opengl ffmpeg > /dev/null 2>&1
!pip install gym[classic_control]
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup gym pyvirtualdisplay tensorflow > /dev/null 2>&1

^C
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/18.3 MB 212.2 kB/s eta 0:01:26

The action is an ndarray with shape (1,) which can take values {0, 1} indicating pushing the cart to the left or right, respectively. Note that the velocity that is reduced or increased by the applied force is not fixed and it depends on the angle the pole is pointing. The center of gravity of the pole varies the amount of energy needed to move the cart underneath it.

In [3]:
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data=''''''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

2023-02-05 20:20:06.939908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-05 20:20:07.223580: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-05 20:20:07.223617: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-05 20:20:08.166176: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Load cartpole env

In [4]:
env = gym.make("CartPole-v1")

Check if there are 2 valid discrete actions that can be performed

In [5]:
print(env.action_space)

Discrete(2)


Display observable space. Note the format of the space being
{
    position
    velocity
    pole angle
    pole angular velocity
}

In [6]:
print(env.observation_space)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [71]:
observation = env.reset()
print("Initial observations:", observation)

Initial observations: (array([ 0.02841219,  0.03179999, -0.04470802, -0.01237144], dtype=float32), {})


In [82]:
observation, reward, truncated, done, info = env.step(0)
print(f"""Position: {observation[0]:2f}, 
Velocity: {observation[0]:2f}, 
Angle: {observation[0]:2f}, 
Angular velocity: {observation[0]:2f}""")
print("Reward for this step:", reward)
print("Is this round done?", done)

Position: -0.178938, 
Velocity: -0.178938, 
Angle: -0.178938, 
Angular velocity: -0.178938
Reward for this step: 0.0
Is this round done? False


Simple game where the policy is to always choose 0.

In [56]:
observation = env.reset()
cumulative_reward = 0
done = False
count = 0
while not done:
    count += 1
    observation, reward, truncated, done, info  = env.step(0)
    cumulative_reward += reward

print(count)
print(f"""Position: {observation[0]:2f}, 
Velocity: {observation[0]:2f}, 
Angle: {observation[0]:2f}, 
Angular velocity: {observation[0]:2f}""")
print("Cumulative reward for this round:", cumulative_reward)

500
Position: -453.750061, 
Velocity: -453.750061, 
Angle: -453.750061, 
Angular velocity: -453.750061
Cumulative reward for this round: 9.0


# Task 1

Development of an RL agent. Demonstrate the correctness of the implementation by sampling a random state from the cart pole environment, inputting to the agent, and outputting a chosen action. Print the values of the state and chosen action in Jupyter notebook.

In [1]:
class RLAgent():
    def __init__(self, env) -> None:
        self.__env = env
        self.__total_reward = 0
        self.__observation, reward, self.__truncated, self.__done, _ = self.__env.reset()
    def get_optimal_action(self):
        pass
    def get_observation(self):
        return self.__observation
    def move(self):
        if (self.__done or self.__truncated):
            raise Exception("Episode already terminated")
        action = self.get_optimal_action()
        self.__observation, reward, self.__truncated, self.__done, _ = self.__env.step(action)
        # update reward
        self.__total_reward += reward
    def isDone(self):
        return self.__done or self.__truncated
    

agent = RLAgent(gym.make("CartPole-v1"))

SyntaxError: incomplete input (38363412.py, line 8)

# Task 2:

Demonstrate the effectiveness of the RL agent. Run for 100 episodes (reset the environment at the beginning of each episode) and plot the cumulative reward against all episodes in Jupyter. Print the average reward over the 100 episodes. The average reward should be larger than 195.

# Task 3:
Render one episode played by the developed RL agent on Jupyter. Please refer to the sample code link for rendering code

# Task 4:

Format the Jupyter notebook by including step-by-step instruction and explanation, such that the notebook is easy to follow and run (refer to the tutorial section in the sample notebook). Include text explanation to demonstrate the originality of your implementation and your understanding of the code. For example, for each task, explain your approach and analyze the output; if you improve an existing approach, explain your improvements.